In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch

from utils.data_utils import preprocessing, custom_train_test_split

clean_df = pd.read_csv("cleaned_data.csv", sep=";")

sns.set_palette("rocket")

## Bins selection

Here we will try to come up with the right divisions for bins, since the data doesn't cover the range quite well

In [2]:
def process_data(df: pd.DataFrame) -> pd.DataFrame:
    df.dropna(subset=['price_pln', 'description'], inplace=True)
    df['price_pln'] = pd.to_numeric(
        df['price_pln'].astype(str).str.replace(r'[^0-9\.-]', '', regex=True),
        errors='coerce'
    )
    
    return df

In [3]:
data = process_data(clean_df)

In [4]:
data.shape

(23752, 22)

In [5]:
print(f'Price below 10k: {len(data[data['price_pln'] < 10000])}')
print(f"Price between 10k–20k: {len(data[(data['price_pln'] > 10000) & (data['price_pln'] < 20000)])}")
print(f"Price between 20k–50k: {len(data[(data['price_pln'] > 20000) & (data['price_pln'] < 50000)])}")
print(f"Price between 50k–75k: {len(data[(data['price_pln'] > 50000) & (data['price_pln'] < 75000)])}")
print(f"Price between 75k–100k: {len(data[(data['price_pln'] > 75000) & (data['price_pln'] < 100000)])}")
print(f"Price between 100k–200k: {len(data[(data['price_pln'] > 100000) & (data['price_pln'] < 200000)])}")
print(f'Price above 200k: {len(data[data['price_pln'] > 200000])}')

Price below 10k: 392
Price between 10k–20k: 1448
Price between 20k–50k: 7125
Price between 50k–75k: 3965
Price between 75k–100k: 2241
Price between 100k–200k: 4158
Price above 200k: 4315


Based on the results above, we opted for the following ranges:

- Below 30k
- Between 30k and 50k
- Between 50k and 75k
- Between 75k and 120k
- Between 120k and 200k
- Above 200k

In [6]:
print(f'Price below 30k: {len(data[data['price_pln'] < 30000])}')
print(f"Price between 30k–50k: {len(data[(data['price_pln'] > 30000) & (data['price_pln'] < 50000)])}")
print(f"Price between 50k–75k: {len(data[(data['price_pln'] > 50000) & (data['price_pln'] < 75000)])}")
print(f"Price between 75k–120k: {len(data[(data['price_pln'] > 75000) & (data['price_pln'] < 120000)])}")
print(f"Price between 120k–200k: {len(data[(data['price_pln'] > 120000) & (data['price_pln'] < 200000)])}")
print(f'Price above 200k: {len(data[data['price_pln'] > 200000])}')

Price below 30k: 4100
Price between 30k–50k: 4857
Price between 50k–75k: 3965
Price between 75k–120k: 3185
Price between 120k–200k: 3208
Price above 200k: 4315


In [7]:
bins = [
    0, 30_000, 50_000, 75_000, 120_000, 200_000, 1_000_000
]

labels = [0, 1, 2, 3, 4, 5]

In [8]:
data['price_class'] = pd.cut(
    data['price_pln'],
    bins=bins,
    labels=labels,
    right=False,
    include_lowest=True
)

In [9]:
print("Class Distribution")
print(data['price_class'].value_counts().sort_index())

Class Distribution
price_class
0    4100
1    4900
2    3988
3    3226
4    3217
5    1670
Name: count, dtype: int64


In [10]:
class_map = {
    0: '< 30k', 1: '30k-50k', 2: '50k-75k', 3: '75k-120k',
    4: '120k-200k', 5: '> 100k'
}

data['class_label'] = data['price_class'].map(class_map)
print("\nExample")
print(data[['price_pln', 'class_label', 'description']].head())


Example
   price_pln class_label                                        description
0    9814170         NaN  grupa tandem\nul. zuzanny 36\n41-219 sosnowiec...
1      37900     30k-50k  peugeot boxer l4h2 2,2 hdi 165mk klimatyzacja ...
2     196500   120k-200k  mercedes-benz grupa wróbel\nzapraszamy do nasz...
3      87945    75k-120k  autoryzowany salon marki opel, firma auto diug...
4      47900     30k-50k  nasza aktualna oferta dostępna na stronie:\n\n...


In [11]:
data.dropna(subset=['price_class'], inplace=True)
df_classification_herbert = data[['description', 'price_class']].copy()

### Model Training

In [12]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


In [13]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [14]:
MODEL_NAME = "allegro/herbert-base-cased"
NUM_LABELS = 6

train_df, eval_df = train_test_split(
    df_classification_herbert,
    test_size=0.2, 
    stratify=df_classification_herbert['price_class'],
    random_state=42
)

train_dataset = Dataset.from_pandas(train_df).remove_columns(['__index_level_0__'])
eval_dataset = Dataset.from_pandas(eval_df).remove_columns(['__index_level_0__'])

train_dataset = train_dataset.rename_column("price_class", "labels")
eval_dataset = eval_dataset.rename_column("price_class", "labels")

print(f"Zbiór treningowy (wiersze): {len(train_dataset)}")
print(f"Zbiór walidacyjny (wiersze): {len(eval_dataset)}")

Zbiór treningowy (wiersze): 16880
Zbiór walidacyjny (wiersze): 4221


In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["description"], truncation=True, padding="max_length")

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    ignore_mismatched_sizes=True
)

Map:   0%|          | 0/16880 [00:00<?, ? examples/s]

Map:   0%|          | 0/4221 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    f1_weighted = f1_score(labels, predictions, average="weighted")
    accuracy = accuracy_score(labels, predictions)

    return {
        "accuracy": accuracy,
        "f1_weighted": f1_weighted,
    }

In [20]:
training_args = TrainingArguments(
    output_dir="./herbert_car_price_model", 
    num_train_epochs=3, 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    eval_strategy="epoch",
    logging_dir='./logs',
    logging_steps=100,
    save_strategy="epoch",
    load_best_model_at_end=True, 
    metric_for_best_model="f1_weighted", 
    fp16=False,
    no_cuda=True
)

/home/mz/Projects/adm/Otomoto_data_mining/.venv/lib/python3.13/site-packages/transformers/training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
print("Rozpoczęcie Fine-Tuning modelu HERBERT...")
train_results = trainer.train()

eval_results = trainer.evaluate()
print("\nWyniki walidacji po Fine-Tuning:")
print(eval_results)

model_save_path = "./herbert_price_classifier"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"\nModel i tokenizer zapisane w: {model_save_path}")

Rozpoczęcie Fine-Tuning modelu HERBERT...


Epoch,Training Loss,Validation Loss


## Baseline Model

In [ ]:
print("PYTORCH VERSION:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("MPS available:", torch.backends.mps.is_available())
print("MPS built:", torch.backends.mps.is_built())

if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current CUDA device:", torch.cuda.current_device())
    print("CUDA device name:", torch.cuda.get_device_name(0))

if torch.backends.mps.is_available():
    print("Using device: MPS (Apple Silicon GPU)")
else:
    print("Using device: CPU")
    
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

device

In [ ]:
price_ranges = [
    0,
    3_000,
    5_000,
    10_000,
    20_000,
    50_000,
    80_000,
    100_000,
    150_000,
    200_000,
    300_000,
    400_000,
    500_000,
    600_000,
    700_000,
    800_000,
    4_000_000,
]
df = clean_df.drop(
    [
        "url",
        "color",
        "posted_date",
        "price_net_info",
        "location",
        "price",
        "country_of_origin",
    ],
    axis=1,
    errors="ignore",
)

df["capacity"] = (
    df["capacity"]
    .str.replace(" cm3", "", regex=False)
    .str.replace(" ", "")
    .astype(float)
)

df["power"] = (
    df["power"].str.replace(" ", "").str.replace("KM", "", regex=False).astype(float)
)

df["mileage"] = (
    df["mileage"].str.replace(" km", "", regex=False).str.replace(" ", "").astype(float)
)

df["price_pln"] = (
    df["price_pln"].str.replace(" ", "").str.replace(",", ".").astype(float)
)

df = df.dropna(
    subset=[
        "model",
        "condition",
        "fuel",
        "brand",
        "body_type",
        "accident_free",
        "year",
        "capacity",
        "power",
        "mileage",
        "seats",
        "description",
    ]
)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


price_ranges = [
    0,
    3_000,
    5_000,
    10_000,
    20_000,
    50_000,
    80_000,
    100_000,
    150_000,
    200_000,
    300_000,
    400_000,
    500_000,
    600_000,
    700_000,
    800_000,
    4_000_000,
]

model_name = "allegro/herbert-klej-cased-v1"

tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(price_ranges)
)

model.to(device)

In [ ]:
class PriceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx]) for key, val in self.encodings.items()
        } | {
            "labels": torch.tensor(self.labels[idx])   # ← here labels[idx] is str
        }
        
    def __len__(self):
        return len(self.labels)

In [ ]:
from sklearn.model_selection import train_test_split


X = df["description"].tolist()
y = pd.cut(df["price_pln"], bins=price_ranges, labels=False).astype(int).tolist()



X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

train_enc = tokenizer(X_train, truncation=True, padding=True, max_length=256)
test_enc = tokenizer(X_test, truncation=True, padding=True, max_length=256)

In [ ]:
from transformers import Trainer, TrainingArguments

train_dataset = PriceDataset(train_enc, y_train)
test_dataset = PriceDataset(test_enc, y_test)

training_args = TrainingArguments(
    output_dir='./model-training',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

In [ ]:
pred = trainer.predict(test_dataset)
pred_class = pred.predictions.argmax(axis=1)
print("Accuracy:", (pred_class == y_test).mean())

In [ ]:
def predict(text):
    # tokenize a single string
    enc = tokenizer(
        [text],
        truncation=True,
        padding=True,
        max_length=256,
        return_tensors="pt"
    )

    with torch.no_grad():
        outputs = model(**enc.to(model.device))
        probs = outputs.logits.softmax(dim=1)
        cls_id = probs.argmax(dim=1).item()
    

    return price_ranges[cls_id]

predict("MERCEDES SPRINTER 2.2CDI 164KM Z 2020 ROKU PRODUKCJI Z PRZEBIEGIEM 168TYS KM Z AUTOMATYCZNĄ SKRZYNIĄ BIEGÓW")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


print("accuracy:", accuracy_score(y_test, pred_class))
print("\nreport:\n", classification_report(y_test, pred_class))

cm = confusion_matrix(y_test, pred_class)
sns.heatmap(cm, annot=True, fmt="d")
plt.xlabel("predicted")
plt.ylabel("true")
plt.show()